# (16.0) Описание решаемой задачи

В окружении выявленного айсберга необходимо осуществить поиск заявок, которые потенциально могут быть элементами цепочки всплытия айсберга, но при исполнении не были выявлены как айсберг, так как сделка была произведена ровно на объем видимой части

# (16.1) Установка всех необходимых пакетов¶

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

# (16.2) Определение функций¶

In [2]:
# перевод времени в формат datetime
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# восстановление стакана заявок и расчет основных метрик
def GlassBuild(for_lob):
    
    #формируем очередь заявок
    
    # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mkt_orders = for_lob['PRICE']!=0
    # таблица выставленных заявок
    plc_orders = for_lob[['ORDERNO','BUYSELL','PRICE','VOLUME']][(for_lob ['ACTION']==1) & no_mkt_orders]
    # таблица отмененных заявок
    wdr_orders = for_lob[['ORDERNO','VOLUME']][(for_lob ['ACTION'] == 0) & no_mkt_orders]
    # таблица исполненных заявок
    trd_orders = for_lob [['ORDERNO','VOLUME']][(for_lob['ACTION']==2) & no_mkt_orders]
    trd_orders = trd_orders.groupby(['ORDERNO']).sum()
    trd_orders['ORDERNO'] = trd_orders.index
    trd_orders.index=range(trd_orders.shape[0])
    # собираем все типы заявок в один датафрейм - очередь заявок
    lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
    lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
    lob_almost = lob_almost.fillna(0)
    lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
    # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]
    
    # строим стакан
    
    lob = lob_almost.pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    
    # считаем метрики ликвидности
    
    # расчет лучшей цены на покупку
    bid_price = lob[lob['B'] > 0].index[0]
    # расчет объема на уровне лучшей цены на покупку
    bid_volume = lob['B'][bid_price]
    # расчет общего объема на стороне покупки
    bid_depth = lob['B'].sum()
    # расчет объема на уровне 10 лучших цен на стороне покупки
    b_10 = lob[lob['B'] > 0].index[0:10]
    volume_b_10 = 0
    for i in b_10:
        volume_b_10 = volume_b_10 + lob['B'][i]
    # расчет лучшей цены на продажу
    ask_price = lob[lob['S'] > 0].index[-1]
    # расчет объема на уровне лучшей цены на продажу
    ask_volume = lob['S'][ask_price]
    # расчет общего объема на стороне продажи
    ask_depth = lob['S'].sum()
    # расчет объема на уровне 10 лучших цен на стороне продажи
    s_10 = lob[lob['S'] > 0].index[-10:]
    volume_s_10 = 0
    for i in s_10:
        volume_s_10 = volume_s_10 + lob['S'][i]
    
    liquidity = [bid_price, bid_volume, bid_depth, volume_b_10, ask_price, ask_volume, ask_depth, volume_s_10]
     
    return(liquidity)

# проверка, что время выставления каждой следующей заявки в цепочке больше времени исполнения предыдущей, если нет, то такая пара исключается из цепочки
def ChainPair(list_of_orderno):
    
    list_of_pairs = []
    
    for i in range(len(list_of_orderno)-1):
    
        cur_orderno = list_of_orderno[i]
        prev_orderno = list_of_orderno[i+1]
        cur_time_place_1 = orders_instr.index[(orders_instr.ORDERNO == cur_orderno)&(orders_instr.ACTION == 1)].to_list()[0]
        cur_time_place = orders_instr.loc[cur_time_place_1,'TIME']
        prev_time_trd_1 = orders_instr.index[(orders_instr.ORDERNO == prev_orderno)&(orders_instr.ACTION == 2)].to_list()[-1]
        prev_time_trd = orders_instr.loc[prev_time_trd_1,'TIME']
    
        if cur_time_place >= prev_time_trd:

            list_of_pairs.append([cur_orderno, prev_orderno])
        
    return(list_of_pairs)

# (16.3) Список всех используемых датафреймов

In [ ]:
# orders - ордерлог на заданную дату
# iceberg_all - список всех найденных айсбергов
# orders_instr - ордерлог по инструменту
# ice_instr - список айсбергов по одному инструменту за один день

# (16.4) Определение входных параметров

In [3]:
# создаем список из всех названий файлов с данными (из них в дальнейшем можно извлечь дату)
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [4]:
# создаем список из инструментов
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [288]:
d = orderlogs[0] 
date = int(d[8:16]) #20190304
ticker = tickers[0] #GAZP
delta = dt.timedelta(microseconds=90000)

# (16.5) Чтение данных

In [18]:
%%time
# преобразуем time в формат datetime
# orders - ордерлог за день
# instr_orders - ордерлог на заданную дату по выбранному инструменту на момет времени t
orders = pd.read_csv(d, header = 0)
orders['together_time'] = str(date)+orders['TIME'].apply(str)
orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
del orders['together_time']
orders_instr = orders[orders.SECCODE == ticker].reset_index()

CPU times: user 6min 3s, sys: 6.35 s, total: 6min 10s
Wall time: 39min 51s


In [76]:
# читаем список выявленных айсберг-заявок + преобразуем время в формат datetime
# iceberg_all - список всех найденных айсбергов
# ice_instr - список айсбергов по одному инструменту за один день
iceberg_all = pd.read_csv('iceberg.csv', header = 0)
iceberg_all['together_time'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME'].apply(str)
iceberg_all['together_time_identified'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME_IDENT'].apply(str)
iceberg_all['TIME'] = [ReverseToDateTime(iceberg_all.together_time[i]) for i in range (len(iceberg_all.together_time))]
iceberg_all['TIME_IDENT'] = [ReverseToDateTime(iceberg_all.together_time_identified[i]) for i in range (len(iceberg_all.together_time_identified))]
iceberg_all['DELTA'] = iceberg_all['TIME_IDENT'] - iceberg_all['TIME']
del iceberg_all['together_time']
del iceberg_all['together_time_identified']
ice_instr = iceberg_all[(iceberg_all.SECCODE == ticker) & (iceberg_all.DATE == date)]

ice_instr.head(50)

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA
174,0,B,20190304,1,47392,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-920,1000,00:00:00
175,1,B,20190304,1,47393,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-120,80,00:00:00
176,2,B,20190304,1,47394,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-2630,880,00:00:00
177,3,B,20190304,1,47395,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-1130,370,00:00:00
178,4,B,20190304,1,47396,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-70,870,00:00:00
179,5,B,20190304,1,47397,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-1000,930,00:00:00
180,6,S,20190304,1,101968,207.00,SBER,2019-03-04 10:01:59.677124,2019-03-04 10:01:59.677124,-3470,5000,00:00:00
181,7,S,20190304,1,101969,207.00,SBER,2019-03-04 10:01:59.677124,2019-03-04 10:01:59.677124,-1550,1530,00:00:00
182,8,S,20190304,1,102941,207.00,SBER,2019-03-04 10:02:01.144606,2019-03-04 10:02:01.677582,-2390,5000,00:00:00.532976
183,9,S,20190304,1,104577,207.00,SBER,2019-03-04 10:02:04.478353,2019-03-04 10:02:20.504241,-2920,5000,00:00:16.025888


# (16.6) Выбираем айсберг для анализа

In [448]:
# вводим индекс айсберга из списка его окружения и извлекаем основную информацию из него
id = 164
test = ice_instr.loc[id]
buysell = ice_instr.loc[id][1]
date = ice_instr.loc[id][2]
orderno = ice_instr.loc[id][4]
price = ice_instr.loc[id][5]
ticker = ice_instr.loc[id][6]
time_place = ice_instr.loc[id][7]
time_ident = ice_instr.loc[id][8]
volume_ini = ice_instr.loc[id][10]

test

index                                164
BUYSELL                                B
DATE                            20190304
ICEBERG                                1
ORDERNO                          2391577
PRICE                                155
SECCODE                             GAZP
TIME          2019-03-04 18:02:41.726467
TIME_IDENT    2019-03-04 18:02:41.756361
VOLUME                             -1190
VOLUME_INI                          1000
DELTA             0 days 00:00:00.029894
Name: 164, dtype: object

# (16.7) Анализ интервала перед выставлением айсберга для поиска потенциальных элементов микроцепочки

In [378]:
# описание айсберга, который был выбран для анализа
ice_instr.loc[id]

index                                 83
BUYSELL                                S
DATE                            20190304
ICEBERG                                1
ORDERNO                          1791777
PRICE                             155.09
SECCODE                             GAZP
TIME          2019-03-04 15:43:06.296113
TIME_IDENT    2019-03-04 15:43:06.296485
VOLUME                              -640
VOLUME_INI                          1000
DELTA             0 days 00:00:00.000372
Name: 83, dtype: object

Предполагается, что найденный айсберг - это элемент цепочки всплытия айсберга и перед ним была исполнена как минимум одна заявка на объем видимой части, которая и инициировала всплытие айсберга. Поиск таких заявок необходимо осуществлять на интервале, когда текущая лучшая цена была не хуже цены айсберга:    
Для стороны 'B' такая цена не должна быть ниже цены айсберга  
Для стороны 'S' такая цена не должна быть выше цены айсберга

In [449]:
%%time

ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 1)&(orders_instr.TIME == time_place)].to_list()[0]
cur_price = orders_instr.loc[ice_index,'PRICE']

for i in range(ice_index):
        
        cur_index = ice_index - i
        for_lob = orders_instr[orders_instr.index <= cur_index]

        if buysell == 'B':
            
            if GlassBuild(for_lob)[0] < cur_price:

                left_border = orders_instr.loc[cur_index,'TIME']
                left_index = cur_index
                
                break

        elif buysell == 'S':
            
            if GlassBuild(for_lob)[4] > cur_price:
                
                left_border = orders_instr.loc[cur_index,'TIME']
                left_index = cur_index
                 
                break

print(left_border, left_index)

2019-03-04 18:02:41.724903 315287
CPU times: user 785 ms, sys: 160 ms, total: 946 ms
Wall time: 684 ms


In [450]:
# выбираем минимум из двух левых границ (определенный пользователем лаг между выставлением и всплытием или найденный период сохранения неизменной цены)
if left_border <= (time_place - delta):
    
    left_border = (time_place - delta)

left_border

Timestamp('2019-03-04 18:02:41.724903')

In [451]:
left_border = (time_place - delta)

Общие условия отбора:  
1. orders_instr.ORDERNO < orderno  
2. orders_instr.PRICE == price
3. orders_instr.BUYSELL == buysell
4. orders_instr.ACTION == 2
5. orders_instr.TIME >= left_border
6. orders_instr.TIME <= time_ident
7. объем выставленной заявки == объему исполнения (НО! надо найти заявку с ACTION = 1 для этого)

In [452]:
# отбираем все уникальные номера заявок по инструменту с соответствующими параметрами, которые попали в заданный интервал перед вскрытием айсберга
test = orders_instr[((orders_instr.BUYSELL == buysell) & 
             (orders_instr.PRICE == price) & 
             (orders_instr.TIME >= left_border)& 
             (orders_instr.TIME <= time_ident)&
             (orders_instr.ORDERNO < orderno)&
             (orders_instr.ACTION == 2)     
            )]['ORDERNO'].unique().tolist()

test

[2391433, 2391462, 2391466, 2391473, 2391493, 2391497, 2391513, 2391518]

In [453]:
# проверяем, что исходный объем найденных заявок совпадает с видимой частью айсберга
test1 = []

for i in test:
    
    trade_orderno = i
    order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]
    
    if order_volume == volume_ini:
        
        test1.append(i)

test1

[2391433, 2391462, 2391466, 2391493, 2391518]

In [454]:
# дописываем номер айсберг заявки в найденную цепочку
test1.append(orderno)
# разворачиваем список заявок
test1 = test1[::-1]

In [455]:
test1

[2391577, 2391518, 2391493, 2391466, 2391462, 2391433]

In [456]:
# пересчитываем цепочку до тех пор, пока из нее не будут исключены все элементы, которые не удовлетворяют
# условию: время выставления каждой следующей должно быть больше времени исполнения предыдущей 
chain = []

while len(test1) != (len(chain)+1):
        
    chain = ChainPair(test1)
    test1 = [chain[0][0]]
    
    for i in chain:
    
        test1.append(i[1])
        
test1

[2391577, 2391518, 2391493, 2391466, 2391462, 2391433]

In [457]:
orders_instr[orders_instr['ORDERNO'].isin (test1)]

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
315227,4953128,4953129,GAZP,B,2019-03-04 18:02:41.632828,2391433,1,155.0,1000,NaN,NaN
315228,4953129,4953130,GAZP,B,2019-03-04 18:02:41.632828,2391433,2,155.0,200,2.936006e+09,155.0
315232,4953133,4953134,GAZP,B,2019-03-04 18:02:41.633412,2391433,2,155.0,600,2.936006e+09,155.0
315240,4953191,4953192,GAZP,B,2019-03-04 18:02:41.668931,2391433,2,155.0,200,2.936006e+09,155.0
315241,4953192,4953193,GAZP,B,2019-03-04 18:02:41.668931,2391462,1,155.0,1000,NaN,NaN
315244,4953195,4953196,GAZP,B,2019-03-04 18:02:41.669576,2391462,2,155.0,800,2.936006e+09,155.0
315247,4953198,4953199,GAZP,B,2019-03-04 18:02:41.670098,2391462,2,155.0,200,2.936006e+09,155.0
315250,4953201,4953202,GAZP,B,2019-03-04 18:02:41.670616,2391466,1,155.0,1000,NaN,NaN
315251,4953202,4953203,GAZP,B,2019-03-04 18:02:41.670616,2391466,2,155.0,150,2.936006e+09,155.0
315254,4953205,4953206,GAZP,B,2019-03-04 18:02:41.671373,2391466,2,155.0,700,2.936006e+09,155.0


In [292]:
%%time
# зацикливаем поиск связанных заявок

x3 = []

for i in range(len(ice_instr)):
    
    print(i)
    id = ice_instr.index[i]
    print(id)

    buysell = iceberg_all.loc[id][1]
    date = iceberg_all.loc[id][2]
    orderno = iceberg_all.loc[id][4]
    price = iceberg_all.loc[id][5]
    ticker = iceberg_all.loc[id][6]
    time_place = iceberg_all.loc[id][7]
    time_ident = iceberg_all.loc[id][8]
    volume_ini = iceberg_all.loc[id][10]
    
    ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 1)&(orders_instr.TIME == time_place)].to_list()[0]
    cur_price = price

    '''for i in range(ice_index):

            cur_index = ice_index - i
            for_lob = orders_instr[orders_instr.index <= cur_index]

            if buysell == 'B':

                if GlassBuild(for_lob)[0] < cur_price:

                    left_border = orders_instr.loc[cur_index,'TIME']
                    left_index = cur_index

                    break

            elif buysell == 'S':

                if GlassBuild(for_lob)[4] > cur_price:

                    left_border = orders_instr.loc[cur_index,'TIME']
                    left_index = cur_index

                    break

    if left_border <= (time_place - delta):
    '''
    
    left_border = (time_place - delta)
    
    x2 = orders_instr[((orders_instr.BUYSELL == buysell) & 
             (orders_instr.PRICE == price) & 
             (orders_instr.TIME >= left_border)& 
             (orders_instr.TIME <= time_ident)&
             (orders_instr.ORDERNO < orderno)&
             (orders_instr.ACTION == 2)     
            )]['ORDERNO'].unique().tolist()
    
    test1 = []

    for i in x2:
    
        trade_orderno = i
        order_volume = orders_instr.loc[(orders_instr.ORDERNO == trade_orderno) & (orders_instr.ACTION == 1)]['VOLUME'].values[0]
    
        if order_volume == volume_ini:
        
            test1.append(i)
               
    print(id, len(test1))
    x3.append([id, len(test1)])

0
0
0 0
1
1
1 0
2
2
2 0
3
3
3 0
4
4
4 1
5
5
5 0
6
6
6 0
7
7
7 2
8
8
8 0
9
9
9 0
10
10
10 0
11
11
11 0
12
12
12 0
13
13
13 0
14
14
14 0
15
15
15 0
16
16
16 1
17
17
17 2
18
18
18 0
19
19
19 0
20
20
20 0
21
21
21 0
22
22
22 0
23
23
23 0
24
24
24 0
25
25
25 1
26
26
26 0
27
27
27 0
28
28
28 0
29
29
29 0
30
30
30 0
31
31
31 0
32
32
32 0
33
33
33 1
34
34
34 0
35
35
35 1
36
36
36 0
37
37
37 2
38
38
38 0
39
39
39 0
40
40
40 0
41
41
41 1
42
42
42 1
43
43
43 1
44
44
44 0
45
45
45 0
46
46
46 1
47
47
47 0
48
48
48 0
49
49
49 1
50
50
50 0
51
51
51 1
52
52
52 0
53
53
53 0
54
54
54 1
55
55
55 0
56
56
56 1
57
57
57 0
58
58
58 1
59
59
59 0
60
60
60 1
61
61
61 2
62
62
62 3
63
63
63 0
64
64
64 1
65
65
65 1
66
66
66 0
67
67
67 0
68
68
68 0
69
69
69 0
70
70
70 1
71
71
71 0
72
72
72 0
73
73
73 1
74
74
74 0
75
75
75 0
76
76
76 2
77
77
77 0
78
78
78 2
79
79
79 0
80
80
80 0
81
81
81 0
82
82
82 3
83
83
83 4
84
84
84 0
85
85
85 1
86
86
86 1
87
87
87 0
88
88
88 3
89
89
89 0
90
90
90 0
91
91
91 0
92
92
92 0
93
93
9

In [87]:
ice_instr.index[8]

182

# (16.5) Анализ интервала после вскрытия айсберга для поиска потенциальных элементов микроцепочки



In [184]:
%%time
ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 2)&(orders_instr.TIME == time_ident)].to_list()[0]
cur_price = orders_instr.loc[ice_index,'PRICE']

for i in range(ice_index, len(orders_instr)):
        
        cur_index = i
        for_lob = orders_instr[orders_instr.index <= cur_index]

        if buysell == 'B':
            
            if GlassBuild(for_lob)[0] > cur_price:

                right_border = orders_instr.loc[cur_index,'TIME']
                right_index = cur_index
                
                break

        elif buysell == 'S':
            
            if GlassBuild(for_lob)[4] < cur_price:
                
                right_border = orders_instr.loc[cur_index,'TIME']
                right_index = cur_index
                 
                break

print(right_border, right_index)

2019-03-04 10:00:03.898113 4243
CPU times: user 7.68 s, sys: 84.2 ms, total: 7.77 s
Wall time: 7.61 s


In [183]:
ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 2)&(orders_instr.TIME == time)].to_list()[0]

IndexError: list index out of range

# Проблема задублированности

In [ ]:
перед началом анализа каждой новой айсберг-заявки проверяем, не включает ли она какие-то ранее сформированные цепочки, если да, то надо апдейтить те цепочки, а не создавать новые
